# Azure Demand Forecasting  
## Notebook 03: Feature Engineering and Final Dataset Preparation  
### Milestone 2: Feature Identification, Engineering & Dataset Prep


In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [86]:
# Load cleaned merged dataset from Milestone 1
df = pd.read_csv('../data/processed/cleaned_merged.csv', parse_dates=['date'])
df.sort_values('date', inplace=True)
df.reset_index(drop=True, inplace=True)

print("Data loaded. Sample:")
df.head()

Data loaded. Sample:


,date,region,resource_type,usage_cpu,usage_storage,users_active,economic_index,cloud_market_demand,holiday
0,2023-01-01,East US,VM,88,1959,470,104.97,0.99,1
1,2023-01-01,Southeast Asia,Container,77,1199,470,104.97,0.99,1
2,2023-01-01,Southeast Asia,Storage,76,1582,369,104.97,0.99,1
3,2023-01-01,Southeast Asia,VM,93,1356,248,104.97,0.99,1
4,2023-01-01,North Europe,Storage,51,1715,476,104.97,0.99,1


## Identify Features and Engineer New Features


## seasonality indicators 

In [87]:
# Extract month, quarter and week of year as features
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['weekofyear'] = df['date'].dt.isocalendar().week

# Flag weekends and holidays from existing 'holiday' column (1=holiday/weekend)
df['is_weekend'] = df['date'].dt.weekday >= 5

print("Seasonality and calendar features added.")
df[['date','month','quarter','weekofyear','is_weekend','holiday']].head()


Seasonality and calendar features added.


,date,month,quarter,weekofyear,is_weekend,holiday
0,2023-01-01,1,1,52,True,1
1,2023-01-01,1,1,52,True,1
2,2023-01-01,1,1,52,True,1
3,2023-01-01,1,1,52,True,1
4,2023-01-01,1,1,52,True,1


In [88]:
# Sort for lag computations
df = df.sort_values(['region', 'date'])

# Create lag features for usage_cpu and users_active (lag-1 and lag-7)
df['usage_cpu_lag1'] = df.groupby('region')['usage_cpu'].shift(1)
df['usage_cpu_lag7'] = df.groupby('region')['usage_cpu'].shift(7)
df['users_active_lag1'] = df.groupby('region')['users_active'].shift(1)
df['users_active_lag7'] = df.groupby('region')['users_active'].shift(7)

print("Lagged demand features created.")
df[['date','region','usage_cpu','usage_cpu_lag1','usage_cpu_lag7','users_active','users_active_lag1','users_active_lag7']].head(10)


Lagged demand features created.


,date,region,usage_cpu,usage_cpu_lag1,usage_cpu_lag7,users_active,users_active_lag1,users_active_lag7
0,2023-01-01,East US,88,NaN,NaN,470,NaN,NaN
9,2023-01-01,East US,70,88.0,NaN,414,470.0,NaN
10,2023-01-01,East US,92,70.0,NaN,388,414.0,NaN
18,2023-01-02,East US,86,92.0,NaN,220,388.0,NaN
21,2023-01-02,East US,93,86.0,NaN,330,220.0,NaN
22,2023-01-02,East US,96,93.0,NaN,250,330.0,NaN
33,2023-01-03,East US,57,96.0,NaN,414,250.0,NaN
34,2023-01-03,East US,85,57.0,88.0,470,414.0,470.0
35,2023-01-03,East US,93,85.0,70.0,401,470.0,414.0
44,2023-01-04,East US,57,93.0,92.0,232,401.0,388.0


## Visualize Lagged CPU Usage


In [89]:
print(df.columns)



# Calculate 7-day rolling average for usage_cpu and users_active by region
df['usage_cpu_roll7'] = df.groupby('region')['usage_cpu'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
df['users_active_roll7'] = df.groupby('region')['users_active'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

print("Rolling average features added.")
display(df[['date','region','usage_cpu','usage_cpu_roll7','users_active','users_active_roll7']].head(10))


Index(['date', 'region', 'resource_type', 'usage_cpu', 'usage_storage',
       'users_active', 'economic_index', 'cloud_market_demand', 'holiday',
       'month', 'quarter', 'weekofyear', 'is_weekend', 'usage_cpu_lag1',
       'usage_cpu_lag7', 'users_active_lag1', 'users_active_lag7'],
      dtype='object')
Rolling average features added.


,date,region,usage_cpu,usage_cpu_roll7,users_active,users_active_roll7
0,2023-01-01,East US,88,88.000000,470,470.000000
9,2023-01-01,East US,70,79.000000,414,442.000000
10,2023-01-01,East US,92,83.333333,388,424.000000
18,2023-01-02,East US,86,84.000000,220,373.000000
21,2023-01-02,East US,93,85.800000,330,364.400000
22,2023-01-02,East US,96,87.500000,250,345.333333
33,2023-01-03,East US,57,83.142857,414,355.142857
34,2023-01-03,East US,85,82.714286,470,355.142857
35,2023-01-03,East US,93,86.000000,401,353.285714
44,2023-01-04,East US,57,81.000000,232,331.000000


In [90]:
# One-hot encode 'region' and 'resource_type'
df = pd.get_dummies(df, columns=['region', 'resource_type'], drop_first=True)

print("Categorical variables one-hot encoded.")
df.head()


Categorical variables one-hot encoded.


,date,usage_cpu,usage_storage,users_active,economic_index,cloud_market_demand,holiday,month,quarter,weekofyear,...,usage_cpu_lag7,users_active_lag1,users_active_lag7,usage_cpu_roll7,users_active_roll7,region_North Europe,region_Southeast Asia,region_West US,resource_type_Storage,resource_type_VM
0,2023-01-01,88,1959,470,104.97,0.99,1,1,1,52,...,NaN,NaN,NaN,88.000000,470.0,False,False,False,False,True
9,2023-01-01,70,621,414,104.97,0.99,1,1,1,52,...,NaN,470.0,NaN,79.000000,442.0,False,False,False,False,False
10,2023-01-01,92,1595,388,104.97,0.99,1,1,1,52,...,NaN,414.0,NaN,83.333333,424.0,False,False,False,True,False
18,2023-01-02,86,1146,220,106.48,1.15,0,1,1,1,...,NaN,388.0,NaN,84.000000,373.0,False,False,False,False,False
21,2023-01-02,93,1767,330,106.48,1.15,0,1,1,1,...,NaN,220.0,NaN,85.800000,364.4,False,False,False,True,False


In [91]:
# Drop rows with NaNs created due to lag features
df_final = df.dropna().reset_index(drop=True)

# Save prepared dataset for modeling
output_path = '../data/processed/final_featured_dataset.csv'
df_final.to_csv(output_path, index=False)

print(f"Final dataset prepared and saved at {output_path}")


Final dataset prepared and saved at ../data/processed/final_featured_dataset.csv


In [ ]:
df = df.sort_values('date')
all_days = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
missing_days = all_days.difference(df['date'])
print(f"Missing days: {missing_days}")


df.set_index('date', inplace=True)
weekly_df = df.resample('W-MON').agg({
    'usage_cpu': 'mean',
    'usage_storage': 'mean',
    'users_active': 'sum',
    # include other columns as needed
}).reset_index()

Missing days: DatetimeIndex([], dtype='datetime64[ns]', freq='D')
